In [1]:
import streamlit as st
from langchain_community.llms import Ollama
from retriever import load_hybrid_retriever
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from report_generator import export_report

In [2]:
llm = Ollama(model="llama3", base_url="http://localhost:11434")

# Load prebuilt retriever
retriever = load_hybrid_retriever()

# Prompt
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a legal AI assistant. Use the given context from IPC/Constitution to answer the question.
If the answer is not in the context, say you don't know.

Context:
{context}

Question:
{question}

Answer concisely, but accurately in legal terms.
"""
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

C:\Users\Tanuj\AppData\Local\Temp\ipykernel_6344\1367693712.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3", base_url="http://localhost:11434")
c:\work\deep_research_agent\retriever.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [3]:
query = "what is the punishment for theft under IPC?"

In [4]:
result = qa_chain({"query": query})

C:\Users\Tanuj\AppData\Local\Temp\ipykernel_6344\3834357911.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


In [6]:
result

{'query': 'what is the punishment for theft under IPC?',
 'result': 'According to the given context from IPC, the punishment for theft under Section 380 (Theft in dwelling house, etc.) is "imprisonment of either description for a term which may extend to seven years, and shall also be liable to fine."'}

In [5]:
for doc in result.get("source_documents", []):
        st.write(f"- {doc.metadata.get('source', 'Unknown')}")